# 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'

In [2]:
# 데이터 불러오기
fpopl = pd.read_csv('fpopl.csv') # 행정동별 유동인구
adstrd = pd.read_csv('adstrd_master.csv') # 8자리 행정동 코드 
card = pd.read_csv('card_20200717.csv') # 업종별 결제금액
weather = pd.read_csv('weather.csv', encoding='cp949')

## 1.데이터 설명

행정동별 유동인구 데이터
- 시기: 2020-01-01 ~ 2020-06-16
- base_ymd : 기준연월일
- tmzon_se_code : 24시간대 구분코드
- sexdstn_se_code : 성별 구분코드 (M:남성, F:여성)
- argde_se_code : 5세단위 연령대 구분코드
- astrd_code : 행정동코드
- popltn_cascnt : 유동인구 수

카드 데이터의 일자별 전체 데이터
- 시기: 2020-01-04 ~ 2020-06-14
- receipt_dttm : 기준연월일
- adstrd_code : 가맹점 위치 기준 행정동 코드 -> 모두 11로 시작하고, 서울시 데이터
- adstrd_nm : 가맹점 위치 기준 행정동명
- mrhst_induty_cl_code : 가맹점 업종 코드
- mrhst_induty_cl_nm : 가맹점 업종명 -> 전체 260개
- selng_cascnt : 매출발생건수
- salamt : 매출발생금액

8자리 행정동 코드
- adstrd_cord : 행정동 코드
- adstrd_nm : 행정동명
- brtc_nm : 시도명
- signgu_nm : 시군구명

날씨 데이터
- 지점명: 위치
- 일시 
- 평균기온

In [3]:
card

,receipt_dttm,adstrd_code,adstrd_nm,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt
0,20200104,1174066000,성내3동,7041,약국,463,5843230
1,20200104,1174066000,성내3동,7022,치과의원,33,7835550
2,20200104,1174066000,성내3동,7021,한의원,53,4589800
3,20200104,1174066000,성내3동,7020,의원,339,9267240
4,20200104,1174066000,성내3동,6110,자동차정비,19,4441000
...,...,...,...,...,...,...,...
3713947,20200614,1121586000,구의2동,4004,대형할인점,136,1195640
3713948,20200614,1121584700,자양4동,9010,인테 리어,2,13100
3713949,20200614,1162052500,보라매동,6140,주 차 장,2,14500
3713950,20200614,1135056000,월계1동,9998,기타전문점,1,38000


In [4]:
adstrd

,adstrd_code,adstrd_nm,brtc_nm,signgu_nm
0,11110515.0,청운효자동,서울특별시,종로구
1,11110530.0,사직동,서울특별시,종로구
2,11110540.0,삼청동,서울특별시,종로구
3,11110550.0,부암동,서울특별시,종로구
4,11110560.0,평창동,서울특별시,종로구
...,...,...,...,...
3488,50130580.0,서홍동,제주특별자치도,서귀포시
3489,50130590.0,대륜동,제주특별자치도,서귀포시
3490,50130600.0,대천동,제주특별자치도,서귀포시
3491,50130610.0,중문동,제주특별자치도,서귀포시


In [5]:
weather

,지점명,일시,평균기온(℃)
0,서울,2020-01-01,-2.2
1,서울,2020-01-02,1.0
2,서울,2020-01-03,-0.1
3,서울,2020-01-04,1.2
4,서울,2020-01-05,1.3
...,...,...,...
174,서울,2020-06-26,21.8
175,서울,2020-06-27,24.1
176,서울,2020-06-28,25.2
177,서울,2020-06-29,23.8


In [6]:
fpopl

,base_ymd,tmzon_se_code,sexdstn_se_code,agrde_se_code,adstrd_code,popltn_cascnt
0,20200101,0,M,age_00,11650560,15
1,20200101,0,M,age_00,11590620,6
2,20200101,0,M,age_00,11560710,4
3,20200101,0,M,age_00,11470680,12
4,20200101,0,M,age_00,11350665,6
...,...,...,...,...,...,...
47546438,20200616,23,F,age_70,11500593,38
47546439,20200616,23,M,age_70,11350624,28
47546440,20200616,23,M,age_70,11305534,47
47546441,20200616,23,F,age_70,11500604,50


## 2. 데이터 탐색 및 전처리 

병합처리를 하기 위해 card 데이터의 receipt_dttm와 같은 형태로 변경해 줍니다.

In [7]:
for i in range(0,179):
    weather.일시[i] =''.join(weather.일시[i].split('-'))
weather.일시

c:\users\eoghk\anaconda3\envs\section\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0      20200101
1      20200102
2      20200103
3      20200104
4      20200105
         ...   
174    20200626
175    20200627
176    20200628
177    20200629
178    20200630
Name: 일시, Length: 179, dtype: object

In [8]:
weather.일시 = weather.일시.astype('int64')

In [9]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   지점명      179 non-null    object 
 1   일시       179 non-null    int64  
 2   평균기온(℃)  179 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.3+ KB


In [10]:
card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3713952 entries, 0 to 3713951
Data columns (total 7 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   receipt_dttm          int64 
 1   adstrd_code           int64 
 2   adstrd_nm             object
 3   mrhst_induty_cl_code  int64 
 4   mrhst_induty_cl_nm    object
 5   selng_cascnt          int64 
 6   salamt                int64 
dtypes: int64(5), object(2)
memory usage: 198.3+ MB


weather 데이터와 병합하기 위해 기준이 되는 receipt_dttm의 컬럼이름을 "일시"로 변경해 줍니다.

In [11]:
card =  card.rename(columns = {"receipt_dttm" :"일시" })

In [12]:
fpopl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47546443 entries, 0 to 47546442
Data columns (total 6 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   base_ymd         int64 
 1   tmzon_se_code    int64 
 2   sexdstn_se_code  object
 3   agrde_se_code    object
 4   adstrd_code      int64 
 5   popltn_cascnt    int64 
dtypes: int64(4), object(2)
memory usage: 2.1+ GB


병합처리를 하기위 해 fpopl또한 base_ymd를 "일시"로 변경해 줍니다.

In [13]:
fpopl = fpopl. rename(columns = {"base_ymd": "일시"})

In [14]:
fpopl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47546443 entries, 0 to 47546442
Data columns (total 6 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   일시               int64 
 1   tmzon_se_code    int64 
 2   sexdstn_se_code  object
 3   agrde_se_code    object
 4   adstrd_code      int64 
 5   popltn_cascnt    int64 
dtypes: int64(4), object(2)
memory usage: 2.1+ GB


각각의 데이터에 결측치값이 있는지 확인해 줍니다.

In [15]:
print(card.isna().sum().sum(), weather.isna().sum().sum(),fpopl.isna().sum().sum(),adstrd.isna().sum().sum())

0 0 0 19


adstrd데이터에 있는 결측치값이 세종시에 있는걸 확인할 수 있습니다.
그러나 이번분석에서는 서울시에 대한 분석만 할것이기 때문에 세종시에 대한 결측치값은 무시해 줍니다.

In [29]:
adstrd[adstrd.signgu_nm.isna()]

,adstrd_code,adstrd_nm,brtc_nm,signgu_nm
1153,36110250.0,조치원읍,세종특별자치시,NaN
1154,36110310.0,연기면,세종특별자치시,NaN
1155,36110320.0,연동면,세종특별자치시,NaN
1156,36110330.0,부강면,세종특별자치시,NaN
1157,36110340.0,금남면,세종특별자치시,NaN
1158,36110350.0,장군면,세종특별자치시,NaN
1159,36110360.0,연서면,세종특별자치시,NaN
1160,36110370.0,전의면,세종특별자치시,NaN
1161,36110380.0,전동면,세종특별자치시,NaN
1162,36110390.0,소정면,세종특별자치시,NaN


merge()를 활용하여 card 데이터와 weather데이터를 합쳐 줍니다.

In [46]:
card1 = pd.merge(card,weather)
card1

,일시,adstrd_code,adstrd_nm,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt,지점명,평균기온(℃)
0,20200104,1174066000,성내3동,7041,약국,463,5843230,서울,1.2
1,20200104,1174066000,성내3동,7022,치과의원,33,7835550,서울,1.2
2,20200104,1174066000,성내3동,7021,한의원,53,4589800,서울,1.2
3,20200104,1174066000,성내3동,7020,의원,339,9267240,서울,1.2
4,20200104,1174066000,성내3동,6110,자동차정비,19,4441000,서울,1.2
...,...,...,...,...,...,...,...,...,...
3630644,20200614,1121586000,구의2동,4004,대형할인점,136,1195640,서울,24.0
3630645,20200614,1121584700,자양4동,9010,인테 리어,2,13100,서울,24.0
3630646,20200614,1162052500,보라매동,6140,주 차 장,2,14500,서울,24.0
3630647,20200614,1135056000,월계1동,9998,기타전문점,1,38000,서울,24.0


card데이터의 adstrd_code데이터가 실질적 자치구별 행정동 코드이지만, adstrd데이터의 adstrd_code로도 행정동이 구별되기때문에 같은 기준으로 변경해줍니다.


In [51]:
adstrd = adstrd.drop("adstrd_nm", axis = 1)

In [52]:
card1.adstrd_code = (card1.adstrd_code/100).astype("int64")
adstrd.adstrd_code = adstrd.adstrd_code.astype("int64")
card2 = pd.merge(card1,adstrd, on = 'adstrd_code')
card2

,일시,adstrd_code,adstrd_nm,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt,지점명,평균기온(℃),brtc_nm,signgu_nm
0,20200104,11740660,성내3동,7041,약국,463,5843230,서울,1.2,서울특별시,강동구
1,20200104,11740660,성내3동,7022,치과의원,33,7835550,서울,1.2,서울특별시,강동구
2,20200104,11740660,성내3동,7021,한의원,53,4589800,서울,1.2,서울특별시,강동구
3,20200104,11740660,성내3동,7020,의원,339,9267240,서울,1.2,서울특별시,강동구
4,20200104,11740660,성내3동,6110,자동차정비,19,4441000,서울,1.2,서울특별시,강동구
...,...,...,...,...,...,...,...,...,...,...,...
3630644,20200614,11530800,항동,7041,약국,45,406180,서울,24.0,서울특별시,구로구
3630645,20200614,11530800,항동,8021,스넥,25,242200,서울,24.0,서울특별시,구로구
3630646,20200614,11530800,항동,8004,일식회집,3,70000,서울,24.0,서울특별시,구로구
3630647,20200614,11530800,항동,5105,보습학원,1,560000,서울,24.0,서울특별시,구로구


fpopl의 일별, 행정동별 유동인구수를 구해 card2의 데이터와 병합쳐리 해줍니다.

In [53]:
aa = pd.merge(card2,fpopl.groupby(["일시", "adstrd_code"])["popltn_cascnt"].sum(), on = ["일시", "adstrd_code"])
aa.head(5)

,일시,adstrd_code,adstrd_nm,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt,지점명,평균기온(℃),brtc_nm,signgu_nm,popltn_cascnt
0,20200104,11740660,성내3동,7041,약국,463,5843230,서울,1.2,서울특별시,강동구,205439
1,20200104,11740660,성내3동,7022,치과의원,33,7835550,서울,1.2,서울특별시,강동구,205439
2,20200104,11740660,성내3동,7021,한의원,53,4589800,서울,1.2,서울특별시,강동구,205439
3,20200104,11740660,성내3동,7020,의원,339,9267240,서울,1.2,서울특별시,강동구,205439
4,20200104,11740660,성내3동,6110,자동차정비,19,4441000,서울,1.2,서울특별시,강동구,205439


행정동별 유동인구에서 자치구별 유동인구를 구해줍니다.  

In [57]:
a= aa[['일시', 'adstrd_nm', 'signgu_nm','popltn_cascnt']].drop_duplicates(subset='adstrd_nm').groupby(["일시","signgu_nm"])["popltn_cascnt"].sum()
a = pd.DataFrame(a)
a.reset_index().head()

,일시,signgu_nm,popltn_cascnt
0,20200104,강남구,4226243
1,20200104,강동구,2368727
2,20200104,강북구,996351
3,20200104,강서구,2465692
4,20200104,관악구,1886990


In [59]:
card3 = pd.merge(card2,a, on = ['일시', 'signgu_nm'])
card3

,일시,adstrd_code,adstrd_nm,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt,지점명,평균기온(℃),brtc_nm,signgu_nm,popltn_cascnt
0,20200104,11740660,성내3동,7041,약국,463,5843230,서울,1.2,서울특별시,강동구,2368727
1,20200104,11740660,성내3동,7022,치과의원,33,7835550,서울,1.2,서울특별시,강동구,2368727
2,20200104,11740660,성내3동,7021,한의원,53,4589800,서울,1.2,서울특별시,강동구,2368727
3,20200104,11740660,성내3동,7020,의원,339,9267240,서울,1.2,서울특별시,강동구,2368727
4,20200104,11740660,성내3동,6110,자동차정비,19,4441000,서울,1.2,서울특별시,강동구,2368727
...,...,...,...,...,...,...,...,...,...,...,...,...
13670,20200104,11110515,청운효자동,7041,약국,102,1357660,서울,1.2,서울특별시,종로구,1685580
13671,20200104,11110515,청운효자동,4201,정장,5,543200,서울,1.2,서울특별시,종로구,1685580
13672,20200104,11110515,청운효자동,4020,슈퍼 마켓,351,8656692,서울,1.2,서울특별시,종로구,1685580
13673,20200104,11110515,청운효자동,4010,편 의 점,434,2739350,서울,1.2,서울특별시,종로구,1685580


In [60]:
card4 = card3.drop(["adstrd_code","adstrd_nm", "지점명","adstrd_nm", "brtc_nm"], axis = 1)
card4.head()

,일시,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt,평균기온(℃),signgu_nm,popltn_cascnt
0,20200104,7041,약국,463,5843230,1.2,강동구,2368727
1,20200104,7022,치과의원,33,7835550,1.2,강동구,2368727
2,20200104,7021,한의원,53,4589800,1.2,강동구,2368727
3,20200104,7020,의원,339,9267240,1.2,강동구,2368727
4,20200104,6110,자동차정비,19,4441000,1.2,강동구,2368727


비슷한 유형의 가맹점들을 묶어 줍니다.

In [61]:
card4.mrhst_induty_cl_code = card4.mrhst_induty_cl_code // 1000
card4.head()

,일시,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt,평균기온(℃),signgu_nm,popltn_cascnt
0,20200104,7,약국,463,5843230,1.2,강동구,2368727
1,20200104,7,치과의원,33,7835550,1.2,강동구,2368727
2,20200104,7,한의원,53,4589800,1.2,강동구,2368727
3,20200104,7,의원,339,9267240,1.2,강동구,2368727
4,20200104,6,자동차정비,19,4441000,1.2,강동구,2368727


DataFrame을 보기 쉽게 하기 위해 컬럼이름을 변경해 줍니다.

In [62]:
columns = {"mrhst_induty_cl_code" : "업종코드",
          "mrhst_induty_cl_nm" :"업종명",
          "selng_cascnt" : "매출발생건수",
          "salamt" : "매출발생금액",
          "signgu_nm" :"자치구",
          "popltn_cascnt": "유동인구수"}
card5 = card4.rename(columns= columns)
card5

,일시,업종코드,업종명,매출발생건수,매출발생금액,평균기온(℃),자치구,유동인구수
0,20200104,7,약국,463,5843230,1.2,강동구,2368727
1,20200104,7,치과의원,33,7835550,1.2,강동구,2368727
2,20200104,7,한의원,53,4589800,1.2,강동구,2368727
3,20200104,7,의원,339,9267240,1.2,강동구,2368727
4,20200104,6,자동차정비,19,4441000,1.2,강동구,2368727
...,...,...,...,...,...,...,...,...
13670,20200104,7,약국,102,1357660,1.2,종로구,1685580
13671,20200104,4,정장,5,543200,1.2,종로구,1685580
13672,20200104,4,슈퍼 마켓,351,8656692,1.2,종로구,1685580
13673,20200104,4,편 의 점,434,2739350,1.2,종로구,1685580


In [63]:
card5.describe().T

,count,mean,std,min,25%,50%,75%,max
일시,13675.0,2.020013e+07,7.770685e+01,20200104.0,20200104.0,20200104.0,20200122.0,2.020040e+07
업종코드,13675.0,6.218135e+00,2.213457e+00,1.0,4.0,7.0,8.0,9.000000e+00
매출발생건수,13675.0,3.384195e+02,3.826040e+03,-11.0,12.0,40.0,174.0,2.147110e+05
매출발생금액,13675.0,1.496334e+07,2.436393e+08,-3395100.0,608138.0,1765000.0,4854500.0,1.234291e+10
평균기온(℃),13675.0,2.466589e+00,2.655466e+00,1.2,1.2,1.2,3.0,1.130000e+01
유동인구수,13675.0,1.568336e+06,1.350496e+06,38943.0,46078.0,1685374.0,2368727.0,4.226243e+06


In [64]:
card5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13675 entries, 0 to 13674
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   일시       13675 non-null  int64  
 1   업종코드     13675 non-null  int64  
 2   업종명      13675 non-null  object 
 3   매출발생건수   13675 non-null  int64  
 4   매출발생금액   13675 non-null  int64  
 5   평균기온(℃)  13675 non-null  float64
 6   자치구      13675 non-null  object 
 7   유동인구수    13675 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 961.5+ KB


데이터세트의 크기를 줄여줄 수 있는 데이터는 크기를 줄여 줍니다. 

In [65]:
card5.일시 = card5.일시.astype('int32')
card5.업종코드 = card5.업종코드.astype('int16')
card5.매출발생건수 = card5.매출발생건수.astype('int32')
card5.업종명 = card5.업종명.astype('category')
card5.자치구 = card5.자치구.astype('category')
card5["평균기온(℃)"] =card5["평균기온(℃)"].astype('float16')
card5.유동인구수 = card5.유동인구수.astype('int32')
card5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13675 entries, 0 to 13674
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   일시       13675 non-null  int32   
 1   업종코드     13675 non-null  int16   
 2   업종명      13675 non-null  category
 3   매출발생건수   13675 non-null  int32   
 4   매출발생금액   13675 non-null  int64   
 5   평균기온(℃)  13675 non-null  float16 
 6   자치구      13675 non-null  category
 7   유동인구수    13675 non-null  int32   
dtypes: category(2), float16(1), int16(1), int32(3), int64(1)
memory usage: 480.0 KB


## 2. 분석 포인트

### 각 자치구별 업종 10가지의 산업 매출 분석을 통해  산업 매출  시각화로 표현할 것입니다.
